In [ ]:
!pip install transformers bitsandbytes accelerate safetensors peft

In [1]:
!which python

/p/project/training2411/ramasubramanian1/HDCRS-school-2024/sc_venv_template/venv/bin/python


In [ ]:
import boto3 
import os

AWS_ACCESS_KEY_ID = <Copied over from SSO login>
AWS_SECRET_ACCESS_KEY = <Copied over from SSO login>
AWS_SESSION_TOKEN = <Copied over from SSO login>

BUCKET_NAME = 'workshop-'

USER = os.environ.get('USER')

def generate_federated_session():
    """
    Method to generate federated session to upload the file from HPC to S3 bucket.
    ARGs:
        filename: Upload filename
    Returns: 
        Signed URL for file upload 
    """
    return boto3.session.Session(
            aws_access_key_id=AWS_ACCESS_KEY_ID,
            aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
            aws_session_token=AWS_SESSION_TOKEN
        )

model_filename = f""
config_filename = f"" # please replace this with path of the config file used for fine-tuning.
identifier = ""
model_basename = os.path.basename(model_filename)
config_basename = os.path.basename(config_filename)


session = generate_federated_session()
s3_connector = session.client('s3')
s3_connector.upload_file(model_filename, BUCKET_NAME, f"models/{identifier}_{model_basename}")
s3_connector.upload_file(config_filename, BUCKET_NAME, f"configs/{identifier}_{config_basename}")

In [14]:
json_path = "/p/home/jusers/ramasubramanian1/jureca/course/kumar/llama-recipes/output_dir/adapter_config.json"
import json

with open(json_path, "r") as f:
    lora_config = json.load(f)

In [15]:
import torch

torch.set_default_device("cuda")


print(lora_config)

{'alpha_pattern': {}, 'auto_mapping': None, 'base_model_name_or_path': '/p/project/training2411/kumar/meta-llama/Llama-2-7b-chat-hf', 'bias': 'none', 'fan_in_fan_out': False, 'inference_mode': True, 'init_lora_weights': True, 'layer_replication': None, 'layers_pattern': None, 'layers_to_transform': None, 'loftq_config': {}, 'lora_alpha': 32, 'lora_dropout': 0.05, 'megatron_config': None, 'megatron_core': 'megatron.core', 'modules_to_save': None, 'peft_type': 'LORA', 'r': 8, 'rank_pattern': {}, 'revision': None, 'target_modules': ['v_proj', 'q_proj'], 'task_type': 'CAUSAL_LM', 'use_dora': False, 'use_rslora': False}


In [16]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    LlamaForCausalLM,
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = LlamaForCausalLM.from_pretrained(
    "/p/home/jusers/ramasubramanian1/jureca/course/kumar/meta-llama/Llama-2-7b-chat-hf",
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(
    "/p/home/jusers/ramasubramanian1/jureca/course/kumar/meta-llama/Llama-2-7b-chat-hf",
    quantization_config=bnb_config,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
for param in model.parameters():
    param.requires_grad = False

from peft import LoraConfig, TaskType, get_peft_config, get_peft_model
from transformers import BitsAndBytesConfig, LlamaForCausalLM

peft_config = LoraConfig(**lora_config)
model = get_peft_model(model, peft_config)

In [18]:
from safetensors import safe_open

tensors = {}
with safe_open(
    "/p/home/jusers/ramasubramanian1/jureca/course/kumar/llama-recipes/output_dir/adapter_model.safetensors",
    framework="pt",
    device=0,
) as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k)  # loads the full tensor given a key
        print(k, tensors[k].dtype, tensors[k].shape)  # Uncomment to view

base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight torch.float32 torch.Size([8, 4096])
base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight torch.float32 torch.Size([4096, 8])
base_model.model.model.layers.0.self_attn.v_proj.lora_A.weight torch.float32 torch.Size([8, 4096])
base_model.model.model.layers.0.self_attn.v_proj.lora_B.weight torch.float32 torch.Size([4096, 8])
base_model.model.model.layers.1.self_attn.q_proj.lora_A.weight torch.float32 torch.Size([8, 4096])
base_model.model.model.layers.1.self_attn.q_proj.lora_B.weight torch.float32 torch.Size([4096, 8])
base_model.model.model.layers.1.self_attn.v_proj.lora_A.weight torch.float32 torch.Size([8, 4096])
base_model.model.model.layers.1.self_attn.v_proj.lora_B.weight torch.float32 torch.Size([4096, 8])
base_model.model.model.layers.10.self_attn.q_proj.lora_A.weight torch.float32 torch.Size([8, 4096])
base_model.model.model.layers.10.self_attn.q_proj.lora_B.weight torch.float32 torch.Size([4096, 8])
base_mod

In [19]:
llama_peft_sd = model.state_dict()
for k in llama_peft_sd:
    if "lora" in k and any(module in k for module in peft_config.target_modules):
        llama_peft_sd[k] = tensors[k[:-14]+k[-6:]]

model.load_state_dict(llama_peft_sd)

<All keys matched successfully>

In [20]:
prompt = """Today's date is May 30, 2024. you are given a string with information of an event with time and location. you should extract the information from the given text in json format as shown:
{{"datetime": "yyyy-mm-dd", "location": "str"}} 

here's the text: {text}
json:
"""

# prompt ="""Assume today's date is June 5, 2024. You are given an Input string: {text} convert it to dict string: 
# """

In [21]:
data_file = "examples.txt"
examples = []
# Sample data
with open(data_file, "r") as file:
    for line in file:
        if line:
            examples.append(line.strip())

In [22]:
for example in examples:
    inputs = tokenizer(
        prompt.format(text=example), return_tensors="pt", return_attention_mask=True
    )
    outputs = model.generate(**inputs, max_length=150)
    text = tokenizer.batch_decode(outputs)[0]
    print(example)
    response = text.split("json:")[-1]
    print(response[: response.find("}") + 1])
    print("=======")

On January 15, 2023, display the flooding events in Jakarta.

{"datetime": "2023-01-15", "location": "Jakarta"}
Show images of burn scars in the Colorado Rockies from March 12, 2023.

{
  "datetime": "2023-03-12",
  "location": "Colorado Rockies"
}
Crop types in Sudan as observed on May 20, 2023.

{"datetime": "2023-05-20", "location": "Sudan"}
February 8, 2023: Provide satellite imagery of flooding in Miami.

{
  "datetime": "2023-02-08",
  "location": "Miami"
}
Burn scars in Oregon, noted on April 17, 2023.

{
  "datetime": "2023-04-17",
  "location": "Oregon"
}
July 25, 2023: Identify crop types in Northern India.

{"datetime": "2023-07-25", "location": "Northern India"}
Flooding in central China on June 30, 2023.

{
    "datetime": "2023-06-30",
    "location": "central China"
}
Satellite images of burn scars in Montana from August 14, 2023.

{"datetime": "2023-08-14", "location": "Montana"}
December 5, 2023: Show crop types in Chile.

{"datetime": "2023-12-05", "location": "Chile"